### Defining Class and role to number encoding

In [1]:
class_dct = {'METABOLISM': 0,
             'MOBILOME': 1,
             'INFORMATION STORAGE AND PROCESSING': 2,
             'CELLULAR PROCESSES AND SIGNALING': 3, 'Unknown': 4}
role_dct = {'Chromatin structure and dynamics': 0,
            'Cell wall/membrane/envelope biogenesis': 1,
            'Amino acid transport and metabolism': 2,
            'Transcription': 3,
            'Cell cycle control, cell division, chromosome partitioning': 4,
            'Secondary metabolites biosynthesis, transport and catabolism': 5,
            'Energy production and conversion': 6,
            'Intracellular trafficking, secretion, and vesicular transport': 7,
            'Cytoskeleton': 8,
            'Lipid transport and metabolism': 9,
            'Defense mechanisms': 10,
            'Coenzyme transport and metabolism': 11,
            'Translation, ribosomal structure and biogenesis': 12,
            'Cell motility': 13,
            'Mobilome: prophages, transposons': 14,
            'Signal transduction mechanisms': 15,
            'Nucleotide transport and metabolism': 16,
            'Replication, recombination and repair': 17,
            'Carbohydrate transport and metabolism': 18,
            'Posttranslational modification, protein turnover, chaperones': 19,
            'Extracellular structures': 20,
            'Inorganic ion transport and metabolism': 21,
            'RNA processing and modification': 22, 'Unknown': 23}

In [2]:
import pandas as pd

In [3]:
def mine(bac,df1, num_of_neighbors):
    info = pd.read_csv('./cog_info_table.csv')
    for index, row in bac.iterrows():
        print(index)
        nei = row['E'].split('\t')[:-1]
        for i in range(len(nei)):
            neii = 'COG' + '0' * (4 - len(nei[i])) + nei[i]
            if neii in info['A']:
                continue
            for j in range(len(nei)):
                neij = 'COG' + '0' * (4 - len(nei[j])) + nei[j]
                if neii == neij or (j - i) > num_of_neighbors or (i - j) > num_of_neighbors:
                    continue
                else:
                    neij_class = info.loc[(info['A'] == neij), 'C'].values
                    neij_role = info.loc[(info['A'] == neij), 'D'].values

                    if len(neij_class) > 0:
                        neij_class = neij_class[0]
                    else:
                        neij_class = 'Unknown'
                    if len(neij_role) > 0:
                        neij_role = neij_role[0]
                    else:
                        neij_role = 'Unknown'
                if neij_class not in class_dct.keys():
                    neij_class = 'Unknown'
                if neij_role not in role_dct.keys():
                    neij_role = 'Unknown'

                df1.loc[df1.name == neii, f'class_{neij_class}'] +=1
                df1.loc[df1.name == neii, f'role_{neij_role}'] +=1

### Extracting the information using "mine" function:

In [ ]:
cogs = pd.read_csv('./cogs.csv')
data = []
for k in range(5665):
    name = 'COG' + '0' * (4 - len(str(k))) + str(k)
    if name in cogs.name.values:
        data.append(name)
for i in range(1,8):
    df1 = pd.DataFrame(data, columns=['name'])
    df1['name'] = df1['name'].astype('str')
    for key in class_dct:
        df1[f'class_{key}'] = 0
    for key in role_dct:
        df1[f'role_{key}'] = 0

    bac = pd.read_csv('./cog_words_bac.csv')
    main2(bac,df1, i)
    bac2 = pd.read_csv('./cog_words_plasmid.csv')
    main2(bac2, df1, i)
    df1.to_csv(f'./df_{i}.csv', index=False)